In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import math
from datetime import datetime
import random

root = tk.Tk()
root.title("Hermes – Main Screen")
root.geometry("1000x500")
root.configure(bg='#2c3e50')

all_locations = [
    "Boston", "New York City", "Philadelphia", "Cleveland", 
    "Baltimore", "Washington DC", "Los Angeles", "Austin", "Miami",
    "Tokyo", "London, England", "Paris, France", 
    "Earth Orbit", "Moon", "Mars"
]

distance_map = {
    ("Boston", "New York City"): (215, 4.0),
    ("Boston", "Philadelphia"): (310, 5.5),
    ("Boston", "Cleveland"): (640, 9.5),
    ("Boston", "Baltimore"): (400, 6.5),
    ("Boston", "Washington DC"): (440, 7.0),
    ("New York City", "Philadelphia"): (95, 2.0),
    ("New York City", "Cleveland"): (460, 7.5),
    ("New York City", "Baltimore"): (190, 3.5),
    ("New York City", "Washington DC"): (225, 4.0),
    ("Philadelphia", "Cleveland"): (430, 6.5),
    ("Philadelphia", "Baltimore"): (100, 2.0),
    ("Philadelphia", "Washington DC"): (140, 2.5),
    ("Cleveland", "Baltimore"): (375, 6.0),
    ("Cleveland", "Washington DC"): (375, 6.0),
    ("Baltimore", "Washington DC"): (40, 1.0),
}
flight_map = {
    ("New York City", "Los Angeles"): (2450, 6.0),
    ("New York City", "Austin"): (1750, 4.0),
    ("New York City", "Miami"): (1280, 3.5),
    ("Los Angeles", "Austin"): (1375, 3.5),
    ("Los Angeles", "Miami"): (2340, 5.5),
    ("Austin", "Miami"): (1100, 3.0),
    ("New York City", "Tokyo"): (6750, 14.0),
    ("New York City", "London, England"): (3450, 7.0),
    ("New York City", "Paris, France"): (3625, 7.5),
}
space_map = {
    "Earth Orbit": (250, 0.5),
    "Moon": (238900, 72.0),
    "Mars": (140000000, 259200.0)
}


mission_logs = []
active_trip = {"progress": 0, "eta": 0, "distance": 0, "mode": None}

main_frame = tk.Frame(root, bg='#2c3e50')
main_frame.pack(fill='both', expand=True)

left_panel = tk.Frame(main_frame, bg='#2c3e50', width=200)
left_panel.pack(side='left', fill='y', padx=(20, 10), pady=20)
left_panel.pack_propagate(False)

content_frame = tk.Frame(main_frame, bg='#34495e', width=500)
content_frame.pack(side='left', fill='both', expand=True, padx=(10, 10), pady=20)
content_frame.pack_propagate(False)

right_panel = tk.Frame(main_frame, bg='#2c3e50', width=280)
right_panel.pack(side='right', fill='y', padx=(10, 20), pady=20)
right_panel.pack_propagate(False)


status_title = tk.Label(right_panel, text="System Status",
                        font=('Ubuntu Mono', 18, 'bold'),
                        bg='#2c3e50', fg='#3498db')
status_title.pack(pady=(20, 10))

status_lines = [
    "AI Core: Online",
    "Navigation: Ready",
    "Sensors: Active",
    "Shield Integrity: Nominal",
    "Life Support: Stable",
    "Comms: Linked",
    "Engine Output: Optimal"
]
for line in status_lines:
    label = tk.Label(right_panel, text=line,
                     font=('Ubuntu Mono', 14),
                     bg='#2c3e50', fg='white')
    label.pack(anchor='w', padx=20, pady=6)


def show_page(page_name):
    for widget in content_frame.winfo_children():
        widget.destroy()
    
    if page_name == "Hermes":
        label = tk.Label(content_frame, text="Hermes – Navigation",
                         font=('Ubuntu Mono', 20, 'bold'), 
                         bg='#34495e', fg='white')
        label.pack(pady=(20, 10))

        desc = tk.Label(content_frame, text="Select mode of travel and locations:",
                        font=('Ubuntu Mono', 13), bg='#34495e', fg='#bdc3c7')
        desc.pack(pady=(0, 20))

        
        mode_var = tk.StringVar(value="Car")
        modes = [("Car", "Car"), ("Flight", "Flight"), ("Space", "Space")]
        for text, value in modes:
            tk.Radiobutton(content_frame, text=text, variable=mode_var, value=value,
                           font=('Ubuntu Mono', 12), bg='#34495e', fg='white',
                           selectcolor='#2c3e50', activebackground='#1abc9c').pack(anchor="w", padx=40)

       
        start_var = tk.StringVar()
        dest_var = tk.StringVar()
        tk.Label(content_frame, text="Start Location", font=('Ubuntu Mono', 12),
                 bg='#34495e', fg='white').pack(pady=(10,0))
        start_box = ttk.Combobox(content_frame, values=all_locations, textvariable=start_var, font=('Ubuntu Mono', 12))
        start_box.pack(pady=5, padx=40, fill="x")

        tk.Label(content_frame, text="Destination", font=('Ubuntu Mono', 12),
                 bg='#34495e', fg='white').pack(pady=(10,0))
        dest_box = ttk.Combobox(content_frame, values=all_locations, textvariable=dest_var, font=('Ubuntu Mono', 12))
        dest_box.pack(pady=5, padx=40, fill="x")

        progress_label = tk.Label(content_frame, text="", font=('Ubuntu Mono', 12),
                                  bg='#34495e', fg='white')
        progress_bar = ttk.Progressbar(content_frame, length=300, mode='determinate', maximum=100,
                                       style="aqua.Horizontal.TProgressbar")

        def update_progress():
            if active_trip["eta"] > 0 and active_trip["progress"] < 100:
                active_trip["progress"] += (100 / active_trip["eta"])
                progress_bar["value"] = active_trip["progress"]
                progress_label.config(text=f"Trip Progress: {int(active_trip['progress'])}%")
                content_frame.after(1000, update_progress)

        def start_trip():
            mode = mode_var.get()
            start = start_var.get()
            dest = dest_var.get()
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            status = "In Progress"

            if mode == "Car":
                key = (start, dest)
                if key in distance_map:
                    distance, hours = distance_map[key]
                else:
                    messagebox.showerror("Error", "Invalid Car Route")
                    return
            elif mode == "Flight":
                key = (start, dest)
                if key in flight_map:
                    distance, hours = flight_map[key]
                else:
                    messagebox.showerror("Error", "Invalid Flight Route")
                    return
            elif mode == "Space":
                if dest in space_map:
                    distance, hours = space_map[dest]
                else:
                    messagebox.showerror("Error", "Invalid Space Destination")
                    return
            else:
                return

            mission_logs.append((timestamp, start, dest, distance, hours, status))
            messagebox.showinfo("Trip Started", f"Mode: {mode}\nFrom: {start}\nTo: {dest}\nDistance: {distance} mi\nETA: {hours} hrs")

            
            active_trip.update({"progress": 0, "eta": int(hours * 2), "distance": distance, "mode": mode})
            progress_label.pack(pady=(20,0))
            progress_bar.pack(pady=5)
            update_progress()

        tk.Button(content_frame, text="Start Trip",
                  font=('Ubuntu Mono', 12),
                  bg='#3498db', fg='white',
                  activebackground='#1abc9c',
                  relief="raised", bd=2,
                  command=start_trip).pack(pady=20)

    elif page_name == "Diagnostics":
        label = tk.Label(content_frame, text="Diagnostics Console",
                         font=('Ubuntu Mono', 20, 'bold'), bg='#34495e', fg='white')
        label.pack(pady=(20, 10))

       
        gauge_canvas = tk.Canvas(content_frame, width=300, height=300, bg="#34495e", highlightthickness=0)
        gauge_canvas.pack(pady=20)

        gauge_canvas.create_oval(30, 30, 270, 270, outline="#3498db", width=4)
        for i in range(0, 210, 30):  # tick marks
            angle = math.radians(180 - i)
            x1 = 150 + 100 * math.cos(angle)
            y1 = 150 - 100 * math.sin(angle)
            x2 = 150 + 85 * math.cos(angle)
            y2 = 150 - 85 * math.sin(angle)
            gauge_canvas.create_line(x1, y1, x2, y2, fill="white", width=2)

        gauge_canvas.create_text(150, 260, text="Speed (mph)", font=('Ubuntu Mono', 12), fill="white")
        needle = gauge_canvas.create_line(150, 150, 150, 50, width=4, fill="red")

      
        tk.Label(content_frame, text="Fuel Levels", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        fuel_bar = ttk.Progressbar(content_frame, length=300, style="green.Horizontal.TProgressbar")
        fuel_bar.pack(pady=5)

        tk.Label(content_frame, text="Connection Strength", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        conn_bar = ttk.Progressbar(content_frame, length=300, style="blue.Horizontal.TProgressbar")
        conn_bar.pack(pady=5)

        tk.Label(content_frame, text="Engine Heat", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        heat_bar = ttk.Progressbar(content_frame, length=300, style="red.Horizontal.TProgressbar")
        heat_bar.pack(pady=5)

        tk.Label(content_frame, text="Oxygen Levels", font=('Ubuntu Mono', 14),
                 bg='#34495e', fg='white').pack()
        oxy_bar = ttk.Progressbar(content_frame, length=300, style="aqua.Horizontal.TProgressbar")
        oxy_bar.pack(pady=5)

        def animate_diagnostics():
          
            fuel_bar['value'] = random.randint(80, 100)
            conn_bar['value'] = random.randint(70, 100)
            heat_bar['value'] = random.randint(60, 85)
            oxy_bar['value'] = random.randint(90, 100)

            if active_trip["progress"] < 100:
                speed = (active_trip["progress"] / 100) * 20000
            else:
                speed = 0
            angle = math.radians(180 - (speed / 20000) * 180)
            x = 150 + 85 * math.cos(angle)
            y = 150 - 85 * math.sin(angle)
            gauge_canvas.coords(needle, 150, 150, x, y)

            content_frame.after(2000, animate_diagnostics)

        animate_diagnostics()

    elif page_name == "Mission Logs":
        label = tk.Label(content_frame, text="Mission Logs",
                         font=('Ubuntu Mono', 20, 'bold'), bg='#34495e', fg='white')
        label.pack(pady=(20, 10))

        if mission_logs:
            for log in mission_logs:
                log_text = f"{log[0]} | {log[1]} → {log[2]} | {log[3]} mi | {log[4]} hrs | {log[5]}"
                tk.Label(content_frame, text=log_text, font=('Ubuntu Mono', 12),
                         bg='#34495e', fg='white').pack(anchor="w", padx=20, pady=3)
        else:
            tk.Label(content_frame, text="No missions yet.",
                     font=('Ubuntu Mono', 12), bg='#34495e', fg='#bdc3c7').pack(pady=20)

    elif page_name == "Settings":
        tk.Label(content_frame, text="Settings (coming soon...)", 
                 font=('Ubuntu Mono', 16), bg='#34495e', fg='white').pack(pady=20)


style = ttk.Style()
style.theme_use("clam")
style.configure("aqua.Horizontal.TProgressbar", troughcolor="#2c3e50", background="#1abc9c")
style.configure("green.Horizontal.TProgressbar", troughcolor="#2c3e50", background="lime")
style.configure("blue.Horizontal.TProgressbar", troughcolor="#2c3e50", background="#3498db")
style.configure("red.Horizontal.TProgressbar", troughcolor="#2c3e50", background="#e74c3c")


button_labels = [("Hermes", "Hermes"), ("Diagnostics", "Diagnostics"), ("Mission Logs", "Mission Logs"), ("Settings", "Settings")]
for text, page in button_labels:
    b = tk.Button(left_panel, text=text, font=('Ubuntu Mono', 14),
                  bg='#3498db', fg='white', activebackground='#1abc9c',
                  command=lambda p=page: show_page(p))
    b.pack(fill='x', pady=10)

show_page("Hermes")
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\smanl\anaconda4\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\smanl\anaconda4\Lib\tkinter\__init__.py", line 862, in callit
    func(*args)
  File "C:\Users\smanl\AppData\Local\Temp\ipykernel_9088\4218106920.py", line 231, in animate_diagnostics
    fuel_bar['value'] = random.randint(80, 100)
    ~~~~~~~~^^^^^^^^^
  File "C:\Users\smanl\anaconda4\Lib\tkinter\__init__.py", line 1733, in __setitem__
    self.configure({key: value})
  File "C:\Users\smanl\anaconda4\Lib\tkinter\__init__.py", line 1722, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\smanl\anaconda4\Lib\tkinter\__init__.py", line 1712, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
_tkinter.TclError: invalid command name ".!frame.!frame2.!progressbar2"
